# Evaluation der Embeddings

Kriterien für Beispielfragen:
- Informationen dazu existiert in den Daten 
- in verschiedenen Episoden findet sich etwas zu den Themen 
- verschiedene Themen aus Sport, Wissenschaft, Gesellschaft

In [2]:
import sys
sys.path.append("..")
from segment_ranking.rank_segments import get_most_similar_segments

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from chromadb_connect import get_most_similar_documents_openai, get_most_similar_documents_voyage
import json
from tqdm import tqdm
from db_connect import db_get_df

llm_model = "gpt-3.5-turbo-0301"

## Topical evaluation

In [2]:
get_most_similar_segments("OPENAI", "Frankfurt", 10, 5, False)

,sentence,end,start,filename,segment_id
0,paul ehrlich soll diesen vorwurf zeit seines l...,1326.08,1265.92,paul-ehrlich-mediziner-immunologe-nobelpreistr...,42
1,"Man kann das schon sehen, dass es Städte waren...",342.34,322.32,der-stadtwald-vom-energielieferanten-zum-erhol...,58
2,Dabei sind wir nur ein Wimpernschlag in der Ti...,51.1,25.92,tiefenzeit-vergangenheit-und-zukunft-wiederent...,3
3,Eine Sendung von Marlene Fercher. Frankfurt im...,57.94,37.24,struwwelpeter-poesie-und-schwarze-paedagogik.mp3,8
4,Bis zu ihrem Tod beschäftigt sie sich mit Frag...,1276.98,1258.56,die-architektin-des-sozialen-wohnens-margarete...,217
5,fürstentum frankfurt genannt ein eigener klein...,1093.96,1051.14,aschaffenburg-residenzstadt-zwischen-mainz-und...,41
6,Professor Andriy Portnov lehrt an der Europa -...,247.16,235.3,die-ukraine-seit-1914-ein-staat-sucht-sich-sel...,30
7,In Frankfurt am Main zum Beispiel. Frankfurt i...,346.42,332.12,der-stadtwald-vom-energielieferanten-zum-erhol...,59
8,Er soll den Spuren des Vaters folgen. Leipzig ...,230.48,210.28,johann-wolfgang-von-goethe-mein-leben-ein-einz...,36
9,Er soll den Spuren des Vaters folgen. Leipzig ...,230.5,210.28,johann-wolfgang-von-goethe-mein-leben-ein-einz...,40


In [ ]:
chat = ChatOpenAI(model="gpt-4-0125-preview").bind(
    response_format={"type": "json_object"},
    temperature=0.5,
    seed=111
)
chat

In [13]:
sample_querys = ["Oktoberfest",
                 "Nürnberger Prozesse",
                 "Oper Synfonie",
                 "Tyrannosaurus Rex",
                 "DDR Essen",
                 "Kreislaufsystem",
                 "Tennisprofi"]

querys_topical = [
    "Mauerfall Geschichte",
    "Oktoberfest Tradition",
    "Ruhrgebiet Industrialisierung",
    "Berliner Kultur",
    "Deutsche Romantik",
    "Bauhaus Bewegung",
    "Weimarer Republik",
    "Schwarzwald Mythologie",
    "DDR Alltagsleben",
    "Preußische Geschichte",
    "Brüder Grimm Märchen",
    "Heiliges Römisches Reich",
    "Deutsche Einheit",
    "Nürnberger Prozesse",
    "Hamburg Hafenstadt",
    "Luther Reformation",
    "Berliner Mauer Kunst",
    "Deutsche Philosophen",
    "Bismarck Ära",
    "Deutsche Automobilindustrie",
    "Sächsische Landschaften",
    "Frankfurt Finanzzentrum",
    "Bayern Traditionen",
    "Dresdner Geschichte",
    "Hansebund Handel",
    "Goethe Literatur",
    "Deutsche Volksfeste",
    "Kölner Karneval",
    "Münchner Bierkultur",
    "Ost-West Konflikt"
    ]

querys_questions = [
    "Wann werden Menschen auf dem Mars landen?",
    "Wie wirkt Yoga auf die geistige Gesundheit?",
    "Was verursacht die Ozeanversauerung?",
    "In welchen Bereichen wird virtuelle Realität eingesetzt?",
    "Wer war Tutanchamun?",
    "Was untersucht die kognitive Psychologie?",
    "Was sind die Folgen vulkanischer Aktivität?",
    "Wie hat sich die Jazzmusik entwickelt?",
    "Was sind die ethischen Herausforderungen bei Robotern?",
    "Wie beeinflusst der Klimawandel arktische Ökosysteme?",
    "Was sind die Risiken digitaler Währungen?",
    "Welche Anwendungen hat die Nanotechnologie?",
    "Was verbirgt sich in den Tiefen der Ozeane?",
    "Wie verändert sich die politische Landschaft Afrikas?",
    "Welche Arten von Meditationstechniken gibt es?",
    "Wie funktioniert genetische Bearbeitung?",
    "Wo wird urbaner Gartenbau betrieben?",
    "Wie können Korallenriffe geschützt werden?",
    "Was regelt das Weltraumrecht?",
    "Wie kann man sich vor Cyberangriffen schützen?",
    "Wer sind die Klimaflüchtlinge?",
    "Kann künstliche Intelligenz kreativ sein?",
    "Welche Herausforderungen gibt es beim Bergsteigen?",
    "Was sind die Folgen von Plastikverschmutzung?",
    "Wie lebten Menschen im antiken Griechenland?",
    "Was können wir von Supernovae lernen?",
    "Was sind die Grundprinzipien der indischen Philosophie?",
    "Warum migrieren Tiere?",
    "Warum ist Wildtierkonservierung wichtig?",
    "Wie entwickelt sich Sprache bei Kindern?",
    ]

querys_refinding = [
    "Max Frisch", # Max Frisch - Der Kampf ums Ich
    "Wetterballons", # Wetterballons - Datenspione des Himmels
    "Bayerischer Nationalpark", # Der Nationalpark Bayerischer Wald - Alles Natur!
    "Taiwan", # Taiwan - Ein Land im Konflikt der Mächte
    "Philosophie des Meeres" # Die Philosophie des Meeres - Von Tiefen und Untiefen
    "Mayas", # Die Maya - Mais, Pyramiden und Metropolen
    "Geschichte Augsburg", #Reichsstadt Augsburg - Geschichte eines Aufstiegs
    "Gorbatschow", # Michail Gorbatschow - Ein Generalsekretär als Reformer 
    "Aal", # Der Aal - Ein Fisch voller Rätsel
    "Selbstwert"
]

refindings_answers = [
    "max-frisch-der-kampf-ums-ich-1.mp3",
    "wetterballons-datenspione-des-himmels.mp3",
    "der-nationalpark-bayerischer-wald-alles-natur.mp3",
    "taiwan-ein-land-im-konflikt-der-maechte.mp3",
    "die-philosophie-des-meeres-von-tiefen-und-untiefen.mp3",
    "die-maya-mais-pyramiden-und-metropolen.mp3",
    "reichsstadt-augsburg-geschichte-eines-aufstiegs.mp3",
    "michail-gorbatschow-ein-generalsekretaer-als-reformer-1.mp3",
    "der-aal-ein-fisch-voller-raetsel.mp3",
    "selbstwert-mut-zur-schwaeche.mp3"
]

query_answers = []

embedding_models = ["OPENAI", "MINI_LM", "TF_IDF", "VOYAGE"]
# embedding_models = ["TF_IDF_MINI_LM", "LLAMA"]

segment_amounts = [1, 3, 10]

In [5]:
system_string = """
    Du bewertest die Ausgabe eines Text Retrieval Systems.
    Bewerte, wie gut die gefundenen Textabschnitte zu dem gestellten Thema passen.
    Beachte dazu folgende Dinge und vergebe Punkte:
    - Precision: Der Text enhält keine irrelevanten Informationen, die nicht zum Thema passen (0-5)
    - Cohesion: Die einzelnen Textausschnitte passen Inhaltlich zusammen (0-2)  
    - Uniqueness: Die Textausschnitte wiederholen keine Informationen (0-3)
    Antworte in JSON Format mit zunächst einer Begründung "Reason", die aus einem Satz beseteht.
    Antworte außerdem jeweils mit einem Score als Parameter "Precision", "Cohesion" und "Uniqueness".
"""

user_string = """    
    Textabschnitte: {documents}

    -----------

    Frage: {query}"""

In [6]:
user_template = ChatPromptTemplate.from_template(user_string)
print(user_template.messages[0].prompt)
print(user_template.messages[0].prompt.input_variables)

input_variables=['documents', 'query'] template='    \n    Textabschnitte: {documents}\n\n    -----------\n\n    Frage: {query}'
['documents', 'query']


In [42]:
user_prompt

[HumanMessage(content='    \n    Textabschnitte: stan, dipper, mable, mermando, soos, bill, pacifica, mc Gucket, schwabbel, lady soosan, grenda, candy, ziege, gnom, gideon, zeit-baby, wendy\n\n    -----------\n\n    Frage: Gravity Falls')]

In [7]:
def gpt_evaluate_answer(documents, query):
    user_prompt = user_template.format_messages(documents=documents,query =query)
    res = chat.invoke(
        [
            SystemMessage(
                content=system_string
            ),
            user_prompt[0]
        ]
    )
    return res


In [16]:
res = gpt_evaluate_answer("stan, dipper, mable, mermando, soos, bill, pacifica, mc Gucket, schwabbel, lady soosan, grenda, candy, ziege, gnom, gideon, zeit-baby, wendy",
                    "Gravity Falls")
json.loads(res.content)

{'Reason': 'Die Textabschnitte listen Charaktere und Wesen aus der Serie Gravity Falls auf und sind daher relevant und zusammenhängend.',
 'Precision': 5,
 'Cohesion': 2,
 'Uniqueness': 3}

In [22]:
evaluations = []

In [25]:
for embedding_model in embedding_models:
    for query in tqdm(querys_questions):
        # for segment_amount in segment_amounts:
            segment_amount = 10
            documents = get_most_similar_segments(embedding_model, query, segment_amount, 5, False)
            

            response = gpt_evaluate_answer(documents, query)
            result = json.loads(response.content)

            result["segment_list_json"] = json.dumps([row["sentence"] for _, row in documents.iterrows()])
            result["filename_list_json"] = json.dumps([row["filename"] for _, row in documents.iterrows()])

            result["embedding_model"] = embedding_model
            result["query"] = query
            result["segment_amount"] = segment_amount
            evaluations.append(result)


  0%|          | 0/30 [00:00<?, ?it/s]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


  3%|▎         | 1/30 [00:15<07:16, 15.05s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


  7%|▋         | 2/30 [00:31<07:19, 15.71s/it]

...

 93%|█████████▎| 28/30 [09:52<00:38, 19.29s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


 97%|█████████▋| 29/30 [10:13<00:19, 19.61s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


100%|██████████| 30/30 [05:48<00:00, 11.63s/it]


In [11]:
len(evaluations)

120

In [12]:
evaluations[0]

{'Reason': 'Die Mehrheit der Textabschnitte befasst sich direkt mit dem Thema Mauerfall und der deutschen Wiedervereinigung, jedoch sind einige Abschnitte thematisch nicht direkt zugehörig.',
 'Precision': 4,
 'Cohesion': 1,
 'Uniqueness': 2,
 'segment_list_json': '["Die DDR hat von sofort an ihre Grenzen zum Bundesgebiet und nach Westberlin bis auf weiteres ge\\u00f6ffnet. Wo warst du, als die Mauer fiel? Eine oft gestellte Frage. Und jeder, der die Zeiten damals miterlebte, kann sich ganz genau daran erinnern, wo er war, was er dachte, was er f\\u00fchlte, als die unglaubliche Nachricht kam.", "Der interne Widerstand w\\u00e4chst, auch der Druck von au\\u00dfen. Ende der 1980er Jahre f\\u00e4llt die Mauer. Das Apartheitregime verliert die letzten kalten Krieger an seiner Seite. Die b\\u00fcrokratische Trennung nach Hautfarbe in allen Lebensbereichen verschlingt Unsummen.", "9. November 1989. Die Berliner Mauer f\\u00e4llt. Ohne Gorbatschow w\\u00e4re das nicht m\\u00f6glich geworden.

In [21]:
import pandas as pd

In [14]:
df = pd.DataFrame(evaluations)

In [103]:
df.dtypes

Reason             object
Precision           int64
Cohesion            int64
Uniqueness          int64
embedding_model    object
query              object
segment_amount      int64
dtype: object

In [105]:
df

,Reason,Precision,Cohesion,Uniqueness,embedding_model,query,segment_amount
0,Die Textabschnitte passen gut zum Thema Oktobe...,5,2,3,OPENAI,Oktoberfest,1
1,Die gefundenen Textabschnitte passen gut zum T...,5,2,3,OPENAI,Oktoberfest,3
2,Die gefundenen Textabschnitte passen nur teilw...,3,1,2,OPENAI,Oktoberfest,10
3,Der gefundene Textabschnitt bezieht sich nicht...,0,0,3,OPENAI,Nürnberger Prozesse,1
4,Die gefundenen Textabschnitte passen gut zum T...,4,2,3,OPENAI,Nürnberger Prozesse,3
...,...,...,...,...,...,...,...
58,Die gefundenen Textabschnitte passen gut zum T...,4,2,3,TF_IDF,Kreislaufsystem,3
59,Die gefundenen Textabschnitte passen gut zum T...,5,2,3,TF_IDF,Kreislaufsystem,10
60,Die gefundenen Textabschnitte passen gut zum T...,5,2,3,TF_IDF,Tennisprofi,1
61,Die gefundenen Textabschnitte passen nicht gut...,2,0,3,TF_IDF,Tennisprofi,3


In [18]:
df[df["embedding_model"] == "OPENAI"]["Precision"].sum()

63

In [19]:
df[df["embedding_model"] == "MINI_LM"]["Precision"].sum()

75

In [20]:
df[df["embedding_model"] == "TF_IDF"]["Precision"].sum()

83

In [21]:
df[df["embedding_model"] == "VOYAGE"]["Precision"].sum()

45

In [18]:
from db_connect import db_save_df

In [17]:
db_save_df(df, "evaluation_questions")

In [ ]:
get_most_similar_segments("OPENAI")

## Re-finding Evaluation

In [7]:
df = db_get_df("transcript_sentences")

filenames = df["filename"].unique()

In [8]:
filenames[30:50]

array(['die-ziege-von-der-kuh-des-kleinen-mannes-zur-kaschmirziege.mp3',
       'koerperwaesche-das-richtige-mass-an-hygiene.mp3',
       'die-wechseljahre-wandel-in-der-lebensmitte.mp3',
       'tattoo-piercing-und-koerperschmuck-was-hautzeichen-sagen.mp3',
       'der-kampf-gegen-pocken-die-erste-impfung-der-geschichte-1.mp3',
       'der-marshall-plan-hilfe-zur-selbsthilfe-1.mp3',
       'der-warschauer-pakt-verteidigungsbuendnis-des-ostens.mp3',
       'vincent-van-gogh-begruender-der-modernen-malerei.mp3',
       'regenerative-kulturen-viel-mehr-als-nachhaltig-1.mp3',
       'andy-warhol-pop-oder-art.mp3',
       'die-macht-des-geruechts-fluesterpost-und-nachrede.mp3',
       'das-fruehe-radio-zwischen-aufklaerung-und-propaganda.mp3',
       'der-eiffelturm-ikone-der-moderne.mp3',
       'jugendbewegungen-in-der-weimarer-republik-romantik-und-rebellion-1.mp3',
       'selbstwert-mut-zur-schwaeche.mp3',
       'velo-radl-mountainbike-ein-gefaehrt-und-seine-geschichte-1.mp3',
      

In [9]:
querys_refinding = [
    "Max Frisch", # Max Frisch - Der Kampf ums Ich
    "Wetterballons", # Wetterballons - Datenspione des Himmels
    "Bayerischer Nationalpark", # Der Nationalpark Bayerischer Wald - Alles Natur!
    "Taiwan", # Taiwan - Ein Land im Konflikt der Mächte
    "Philosophie des Meeres" # Die Philosophie des Meeres - Von Tiefen und Untiefen
    "Mayas", # Die Maya - Mais, Pyramiden und Metropolen
    "Geschichte Augsburg", #Reichsstadt Augsburg - Geschichte eines Aufstiegs
    "Gorbatschow", # Michail Gorbatschow - Ein Generalsekretär als Reformer 
    "Aal", # Der Aal - Ein Fisch voller Rätsel
    "Selbstwert"
]

refindings_answers = [
    "max-frisch-der-kampf-ums-ich-1.mp3",
    "wetterballons-datenspione-des-himmels.mp3",
    "der-nationalpark-bayerischer-wald-alles-natur.mp3",
    "taiwan-ein-land-im-konflikt-der-maechte.mp3",
    "die-philosophie-des-meeres-von-tiefen-und-untiefen.mp3",
    "die-maya-mais-pyramiden-und-metropolen.mp3",
    "reichsstadt-augsburg-geschichte-eines-aufstiegs.mp3",
    "michail-gorbatschow-ein-generalsekretaer-als-reformer-1.mp3",
    "der-aal-ein-fisch-voller-raetsel.mp3",
    "selbstwert-mut-zur-schwaeche.mp3"
]

In [25]:
querys_refinding = [
"Kaschmirziege",
"Körperhygiene",
"Wechseljahre",
"Hautschmuck",
"Pockenimpfung",
"Marshall-Plan",
"Warschauer-Pakt",
"Van Gogh",
"Regenerativ",
"Warhol",
"Gerüchte",
"Frühradio",
"Eiffelturm",
"Jugendbewegungen",
"Selbstwert",
"Fahrradgeschichte",
"Kalter Krieg",
"Wassermann",
"Hoegner",
"Dickens",
]

refindings_answers = [
    'die-ziege-von-der-kuh-des-kleinen-mannes-zur-kaschmirziege.mp3',
       'koerperwaesche-das-richtige-mass-an-hygiene.mp3',
       'die-wechseljahre-wandel-in-der-lebensmitte.mp3',
       'tattoo-piercing-und-koerperschmuck-was-hautzeichen-sagen.mp3',
       'der-kampf-gegen-pocken-die-erste-impfung-der-geschichte-1.mp3',
       'der-marshall-plan-hilfe-zur-selbsthilfe-1.mp3',
       'der-warschauer-pakt-verteidigungsbuendnis-des-ostens.mp3',
       'vincent-van-gogh-begruender-der-modernen-malerei.mp3',
       'regenerative-kulturen-viel-mehr-als-nachhaltig-1.mp3',
       'andy-warhol-pop-oder-art.mp3',
       'die-macht-des-geruechts-fluesterpost-und-nachrede.mp3',
       'das-fruehe-radio-zwischen-aufklaerung-und-propaganda.mp3',
       'der-eiffelturm-ikone-der-moderne.mp3',
       'jugendbewegungen-in-der-weimarer-republik-romantik-und-rebellion-1.mp3',
       'selbstwert-mut-zur-schwaeche.mp3',
       'velo-radl-mountainbike-ein-gefaehrt-und-seine-geschichte-1.mp3',
       'deutschland-im-kalten-krieg-opfer-akteur-profiteur.mp3',
       'jakob-wassermann-der-fast-vergessene-weltstar-des-romans.mp3',
       'wilhelm-hoegner-ministerpraesident-auf-befehl-1.mp3',
       'charles-dickens-leben-und-werk.mp3'
]

In [16]:
results = []

In [26]:
for embedding_model in embedding_models:
    for query, answer in tqdm(zip(querys_refinding, refindings_answers)):
        # for segment_amount in segment_amounts:
            segment_amount = 3
            documents = get_most_similar_segments(embedding_model, query, segment_amount, 5, False)
            result = {
                "model": embedding_model,
                "query": query,
                "score": 0
            }
            if answer in documents["filename"][0]:
                result["score"] = 3
            elif answer in documents["filename"][1]:
                result["score"] = 2
            elif answer in documents["filename"][2]:
                result["score"] = 1
            results.append(result)
                
            


20it [01:19,  3.98s/it]
100%|██████████| 434/434 [00:02<00:00, 183.99it/s]
20it [01:56,  5.80s/it]
0it [00:00, ?it/s]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


100%|██████████| 434/434 [00:01<00:00, 281.79it/s]
1it [00:13, 13.46s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


100%|██████████| 434/434 [00:01<00:00, 268.22it/s]
2it [00:26, 13.42s/it]

...

100%|██████████| 434/434 [00:01<00:00, 303.26it/s]
18it [04:18, 13.05s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


100%|██████████| 434/434 [00:01<00:00, 311.00it/s]
19it [04:31, 13.20s/it]

Loading data file - /Users/br/Projects/Bachelorarbeit/data/german.dic


100%|██████████| 434/434 [00:02<00:00, 174.72it/s]
20it [04:48, 14.43s/it]
20it [02:44,  8.22s/it]


In [27]:
df = pd.DataFrame(results)

In [28]:
df.dtypes

model    object
query    object
score     int64
dtype: object

In [29]:
from db_connect import db_append_df

In [30]:
db_append_df(df, "evaluation_re_finding")